<a href="https://colab.research.google.com/github/Tabook22/AI/blob/main/GoodWSD01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install numpy tqdm scikit-learn transformers torch

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
  Using cached nvidia_nvjitlink_cu12-12.4.127-py3-none-m

In [ ]:
import json
import numpy as np
from tqdm import tqdm
from collections import defaultdict
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from transformers import AutoTokenizer, AutoModel
import torch
import time

In [ ]:
# Load CAMeLBERT tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("CAMeL-Lab/bert-base-arabic-camelbert-msa")
model = AutoModel.from_pretrained("CAMeL-Lab/bert-base-arabic-camelbert-msa")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/86.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/468 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/305k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/439M [00:00<?, ?B/s]

In [ ]:
# Function to load JSON data
def load_json(filepath):
    try:
        with open(filepath, 'r', encoding='utf-8') as f:
            return json.load(f)
    except json.JSONDecodeError as e:
        print(f"Error parsing JSON at line {e.lineno}, column {e.colno}: {e.msg}")
        raise

In [ ]:
# Prepare labeled data for training and evaluation
"""
The function takes two parameters:
data: The input data, which is assumed to be a list or iterable containing dictionary entries.
wsd_dict: A dictionary that maps lemma IDs to their corresponding senses and glosses.
"""
def prepare_labeled_data(data, wsd_dict):
    labeled_data = [] #initializes an empty list called labeled_data to store the labeled entries.
    #iterates over each entry in the data ,and extracts the word, lemma_id, and correct_gloss_id from the entry.
    for entry in data:
        word = entry['word']
        lemma_id = entry['lemma_id']
        #The correct_gloss_id is obtained using entry.get('gloss_id'), which returns the value of 'gloss_id' if it exists in the entry, or None otherwise.
        correct_gloss_id = entry.get('gloss_id')

        #For each entry, it retrieves the senses associated with the lemma_id from the wsd_dict.
        #It then iterates over each sense in the retrieved senses: For each sense, it creates a new dictionary entry with the following key-value pairs:
        for sense in wsd_dict[lemma_id]:
            labeled_data.append({
                'context_id': entry['context_id'],
                'context': entry['context'],
                'word': word,
                'lemma_id': lemma_id,
                'gloss_id': sense['gloss_id'],
                'gloss': sense['gloss'],
                'label': 1 if sense['gloss_id'] == correct_gloss_id else 0 # A binary label indicating whether the sense's gloss ID matches the correct gloss ID. It is assigned a value of 1 if they match, and 0 otherwise.
            })
    return labeled_data

In [ ]:
# Extract or fetch cached embeddings for a given text
embedding_cache = defaultdict(lambda: None)

In [ ]:
def encode_text(text):
    tokens = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=512)
    return tokens

def extract_embeddings(text):
    tokens = encode_text(text)
    with torch.no_grad():
        output = model(**tokens)
    return output.last_hidden_state.mean(dim=1).squeeze().numpy()

def extract_embeddings_with_cache(text):
    if embedding_cache[text] is None:
        embedding_cache[text] = extract_embeddings(text)
    return embedding_cache[text]

this code extracts features from the input data by
processing each entry, extracting embeddings for the
'context' and 'gloss', and storing the embeddings along
with the corresponding 'label' in the features list. It also provides progress monitoring every log_interval seconds to display the progress, elapsed time, and estimated remaining time.



This is a function named prepare_features that takes two parameters:


data: The input data to process.

log_interval: The interval (in seconds) at which progress updates should be logged. Default is 60 seconds.


In [ ]:
# Prepare features with progress monitoring every minute
def prepare_features(data, log_interval=60):
    features = [] #an empty list that will store the extracted features, which are the embeddings of the context, and gloss, and assigned label
    start_time = time.time() #records the start time of the feature extraction process.
    last_update_time = start_time #is initialized to the start time and will be used to track the time of the last progress update.

    #This loop iterates over each entry in the data using enumerate to get the index i and the entry itself.
    #tqdm is used to wrap the data to provide a progress bar with a description "Extracting Features" and the unit "pair".
    for i, entry in enumerate(tqdm(data, desc="Extracting Features", unit="pair")):
        context_emb = extract_embeddings_with_cache(entry['context']) #It extracts the embeddings for the 'context' using the extract_embeddings_with_cache function and assigns it to context_emb.
        gloss_emb = extract_embeddings_with_cache(entry['gloss']) #It extracts the embeddings for the 'gloss' using the extract_embeddings_with_cache function and assigns it to gloss_emb.
        label = entry['label'] #It retrieves the 'label' from the entry and assigns it to label.
        features.append((context_emb, gloss_emb, label)) #It appends a tuple (context_emb, gloss_emb, label) to the features list.

        current_time = time.time() #After each iteration It records the current_time.
        if current_time - last_update_time >= log_interval:
            elapsed = current_time - start_time
            progress = (i + 1) / len(data)
            remaining = elapsed / progress - elapsed
            tqdm.write(
                f"Progress: {progress * 100:.2f}% - Elapsed: {elapsed:.2f}s - Remaining: {remaining:.2f}s"
            )
            last_update_time = current_time
    return features #Finally, the function returns the list of extracted features.

In [ ]:
# Concatenate features for classifier input
def concatenate_features(features):
    X, y = [], []
    for context_emb, gloss_emb, label in features:
        X.append(np.concatenate((context_emb, gloss_emb)))
        y.append(label)
    return np.array(X), np.array(y)

In [ ]:
# Load all datasets
"""
train_data = load_json('/content/drive/MyDrive/WSD/Data/train.json') #The training dataset used to train the WSD model.
dev_data = load_json('/content/drive/MyDrive/WSD/Data/dev.json')#The development or validation dataset used for model tuning and intermediate evaluations.
#test_data = load_json('/content/drive/MyDrive/WSD/Data/test.json')#The test dataset used for final evaluation of the trained model.
test_data = load_json('/content/drive/MyDrive/WSD/Data/test2.json')#The test dataset used for final evaluation of the trained model.
wsd_dict_raw = load_json('/content/drive/MyDrive/WSD/Data/WSD_dict.json')#The dictionary containing the mapping between lemma IDs and their corresponding glosses.
"""

train_data = load_json('train.json') #The training dataset used to train the WSD model.
dev_data = load_json('dev.json')#The development or validation dataset used for model tuning and intermediate evaluations.
test_data = load_json('test2.json')#The test dataset used for final evaluation of the trained model.
wsd_dict_raw = load_json('WSD_dict.json')#The dictionary containing the mapping between lemma IDs and their corresponding glosses.


In [ ]:
# Prepare WSD dictionary
wsd_dict = defaultdict(list)
for entry in wsd_dict_raw:
    wsd_dict[entry['lemma_id']].append({
        'gloss_id': entry['gloss_id'],
        'gloss': entry['gloss']
    })

In [ ]:
# Prepare labeled data,  takes the raw data (train_data or dev_data) and the wsd_dict as input and returns labeled data.
# Notice: wsd_dic, is a mapping which provides the information related to Word Sense Disambiguation (WSD). It contains mappings between words, their senses, and corresponding labels or annotations.
train_labeled = prepare_labeled_data(train_data, wsd_dict)
dev_labeled = prepare_labeled_data(dev_data, wsd_dict)

These extracted features can be used for further processing, such as training a machine learning model and evaluating its performance on the development set.


The separation of training and development data is a common practice in machine learning to assess the model's performance on unseen data and prevent overfitting. The training data is used to train the model, while the development data is used to evaluate the model's performance and make necessary adjustments.

In [ ]:
# Extract features for training and development
train_features = prepare_features(train_labeled) #contain the extracted features for the training data, a
dev_features = prepare_features(dev_labeled) #contain the extracted features for the development (validation) data.

Extracting Features:   1%|          | 726/115424 [01:00<49:30, 38.61pair/s]

Progress: 0.63% - Elapsed: 60.01s - Remaining: 9428.92s


Extracting Features:   1%|          | 1380/115424 [02:00<4:44:19,  6.68pair/s]

Progress: 1.20% - Elapsed: 120.08s - Remaining: 9923.51s


Extracting Features:   2%|▏         | 2882/115424 [03:00<12:41, 147.87pair/s]

...

Extracting Features:  95%|█████████▍| 109567/115424 [35:03<00:42, 138.87pair/s]

Progress: 94.90% - Elapsed: 2103.11s - Remaining: 112.97s


Extracting Features:  81%|████████  | 11978/14834 [01:00<00:20, 137.45pair/s]

Progress: 80.75% - Elapsed: 60.05s - Remaining: 14.32s


Extracting Features: 100%|██████████| 14834/14834 [01:21<00:00, 181.55pair/s]


In [ ]:
# Prepare feature matrices
X_train, y_train = concatenate_features(train_features)
X_dev, y_dev = concatenate_features(dev_features)

In [ ]:
# Train a logistic regression classifier with class weight adjustment
class_weights = {0: 1, 1: 2}
clf = LogisticRegression(max_iter=1000, class_weight=class_weights)
clf.fit(X_train, y_train)

LogisticRegression(class_weight={0: 1, 1: 2}, max_iter=1000)

In [ ]:
# Evaluate the model
y_pred = clf.predict(X_dev)
print(classification_report(y_dev, y_pred))

              precision    recall  f1-score   support

           0       0.92      0.82      0.87     12033
           1       0.47      0.68      0.56      2801

    accuracy                           0.79     14834
   macro avg       0.69      0.75      0.71     14834
weighted avg       0.83      0.79      0.81     14834



In [ ]:
# Predict gloss for each word in the test data
def predict_gloss(word_context, glosses):
    context_emb = extract_embeddings_with_cache(word_context)
    scores = []
    for gloss in glosses:
        gloss_emb = extract_embeddings_with_cache(gloss['gloss'])
        concatenated = np.concatenate((context_emb, gloss_emb))
        score = clf.predict_proba([concatenated])[0][1]
        scores.append((gloss['gloss_id'], score))
    return max(scores, key=lambda x: x[1])[0]

In [ ]:
def evaluate_on_test(test_data, wsd_dict):
    correct = 0
    total = len(test_data)

    for entry in test_data:
        word_context = entry['context']
        lemma_id = entry['lemma_id']
        possible_glosses = wsd_dict.get(lemma_id, [])
        if not possible_glosses:
            continue
        predicted_gloss_id = predict_gloss(word_context, possible_glosses)
        correct_gloss_id = entry.get('gloss_id')
        if predicted_gloss_id == correct_gloss_id:
            correct += 1
        else:
            print(f"Context: {word_context}, Predicted Gloss: {predicted_gloss_id}, Correct Gloss: {correct_gloss_id}")

    accuracy = correct / total if total else 0
    return accuracy

In [ ]:
# Print the first few entries of the test data to check their structure
print(test_data[:10])  # Adjust number as necessary to see enough entries

[{'context_id': 'context.25939', 'context': 'الفرق بين إمكانية الشرح وقابلية التفسير لتعلم الآلة', 'word': 'إمكانية', 'lemma_id': 'ar.1446', 'gloss_id': 'gloss.5385'}, {'context_id': 'context.19300', 'context': 'أتقدم بالشكر لرئاسة جامعة تشرين ممثلة', 'word': 'بالشكر', 'lemma_id': 'ar.2754', 'gloss_id': 'gloss.10028'}, {'context_id': 'context.5772', 'context': 'الحل بالطريقة المباشرة', 'word': 'الحل', 'lemma_id': 'ar.1038', 'gloss_id': 'gloss.3648'}, {'context_id': 'context.14169', 'context': 'كما تعدّ ثاني أكبر دولة من حيث عدد السكان بعد جمهوريّة الصين الشعبيّة،', 'word': 'ثاني', 'lemma_id': 'ar.590', 'gloss_id': 'gloss.1666'}, {'context_id': 'context.21908', 'context': 'إن الأداء "هو وسيلة إقناع من خلال إثارة السرور" (Marvin ،', 'word': 'إثارة', 'lemma_id': 'ar.4266', 'gloss_id': 'gloss.14244'}, {'context_id': 'context.17882', 'context': 'فكيف يُنهى العبد عن سؤال الله ما يريده قلبه بدعوى أنه تخصيص ممنوع بل يجب عليه أن يكرر دعاء عاما هو بنفسه لا يعرف ماذا يريد من ربه ؟', 'word': 'الله

In [ ]:
# Evaluate on the test data
test_accuracy = evaluate_on_test(test_data, wsd_dict)
print(f"Test Accuracy: {test_accuracy:.4f}")

Context: حالة موقع تحريج كفردبيل- ريف جبلة ", Predicted Gloss: gloss.15993, Correct Gloss: gloss.15992
Context: وليس للقانون أثر رجعي., Predicted Gloss: gloss.14893, Correct Gloss: gloss.14888
Context: بحسب رئيس جمعية مزارعي فزان " عثمان الطاهر" حيث وصل إنتاج ليبيا من القمح والشعير خلال العام الماضي إلى (100 ) ألف طن،, Predicted Gloss: gloss.4777, Correct Gloss: gloss.4775
Context: - الدرجة مكونة من مرتبتين (احاد وعشرات) مرتبة العشرات تاخذ القيم (6,7,8,9) فمرتبة (6) ظهرت مرة واحدة لان كل درجات مرتبة عشرات (6) اكبر من (65) ،, Predicted Gloss: gloss.5874, Correct Gloss: gloss.5873
Context: حيث يذكر في الباب الأول (حقوق الشعب) نصوصاً قانونية تتعلق بالجنسية العراقية،, Predicted Gloss: gloss.1599, Correct Gloss: gloss.1590
Context: وأضاف ان الشركة عندما بدأت أعمالها كان اهتمامها يقتصر على الأدوات المنزلية الكهربائية من برادات وغسالات وأفران ومكانس كهربائية., Predicted Gloss: gloss.2782, Correct Gloss: gloss.2784
Context: والتي هي من أهم مبادئ حقوق الانسان في وقتنا الحاضر واعادة تعمير ماخربه

**Saving the Model**

After training the logistic regression model (clf), you need to save it to a .pkl file. Here’s how you can do that using Python’s pickle:

In [ ]:
import pickle

# Assuming clf is your trained Logistic Regression model
with open('wsd_model.pkl', 'wb') as f:
    pickle.dump(clf, f)

Creating the Dictionary File

Your WSD system requires a dictionary (wsd_dict.json) that maps lemma IDs to possible glosses. Make sure this dictionary is correctly formatted and contains all necessary information.
To create or update this file, you can serialize your dictionary to JSON:

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Assuming wsd_dict is your dictionary containing the mappings
with open('wsd_dict.json', 'w', encoding='utf-8') as f:
    json.dump(wsd_dict, f, ensure_ascii=False, indent=4)

predicted gloss ID, the correct gloss ID, and the ranking score:

In [ ]:
def predict_gloss(word_context, glosses):
    context_emb = extract_embeddings_with_cache(word_context)
    scores = []
    for gloss in glosses:
        gloss_emb = extract_embeddings_with_cache(gloss['gloss'])
        concatenated = np.concatenate((context_emb, gloss_emb))
        score = clf.predict_proba([concatenated])[0][1]
        scores.append((gloss['gloss_id'], score))
    predicted_gloss_id, highest_score = max(scores, key=lambda x: x[1])
    return predicted_gloss_id, highest_score

In [ ]:
def evaluate_on_test_with_progress_and_debug(test_data, wsd_dict):
    correct = 0
    total = len(test_data)
    start_time = time.time()
    last_update_time = start_time

    for i, entry in enumerate(tqdm(test_data, desc="Evaluating", unit="pair")):
        word_context = entry['context']
        lemma_id = entry['lemma_id']
        possible_glosses = wsd_dict.get(lemma_id, [])
        if not possible_glosses:
            continue
        predicted_gloss_id, ranking_score = predict_gloss(word_context, possible_glosses)
        correct_gloss_id = entry.get('gloss_id')
        if predicted_gloss_id == correct_gloss_id:
            correct += 1
        print(f"Context: {word_context}, Predicted Gloss: {predicted_gloss_id}, Correct Gloss: {correct_gloss_id}, Ranking Score: {ranking_score:.4f}")

        current_time = time.time()
        if current_time - last_update_time >= 60:
            elapsed = current_time - start_time
            progress = (i + 1) / total
            remaining = elapsed / progress - elapsed
            tqdm.write(
                f"Progress: {progress * 100:.2f}% - Elapsed: {elapsed:.2f}s - Remaining: {remaining:.2f}s"
            )
            last_update_time = current_time

    accuracy = correct / total if total else 0
    elapsed_time = time.time() - start_time
    print(f"Evaluation Completed - Elapsed Time: {elapsed_time:.2f}s")
    return accuracy

# Evaluate on the test data with debug prints
test_accuracy = evaluate_on_test_with_progress_and_debug(test_data, wsd_dict)
print(f"Test Accuracy: {test_accuracy:.4f}")

Evaluating:   1%|▏         | 65/4496 [00:00<00:06, 647.19pair/s]

Context: الفرق بين إمكانية الشرح وقابلية التفسير لتعلم الآلة, Predicted Gloss: gloss.5385, Correct Gloss: gloss.5385, Ranking Score: 0.9690
Context: أتقدم بالشكر لرئاسة جامعة تشرين ممثلة, Predicted Gloss: gloss.10028, Correct Gloss: gloss.10028, Ranking Score: 0.6649
Context: الحل بالطريقة المباشرة, Predicted Gloss: gloss.3648, Correct Gloss: gloss.3648, Ranking Score: 0.6752
Context: كما تعدّ ثاني أكبر دولة من حيث عدد السكان بعد جمهوريّة الصين الشعبيّة،, Predicted Gloss: gloss.1666, Correct Gloss: gloss.1666, Ranking Score: 0.7341
Context: إن الأداء "هو وسيلة إقناع من خلال إثارة السرور" (Marvin ،, Predicted Gloss: gloss.14244, Correct Gloss: gloss.14244, Ranking Score: 0.7665
Context: فكيف يُنهى العبد عن سؤال الله ما يريده قلبه بدعوى أنه تخصيص ممنوع بل يجب عليه أن يكرر دعاء عاما هو بنفسه لا يعرف ماذا يريد من ربه ؟, Predicted Gloss: gloss.1826, Correct Gloss: gloss.1826, Ranking Score: 0.7445
Context: مـن أهمّهـا روايتـه “فـي الحشـد”., Predicted Gloss: gloss.10303, Correct Gloss: gloss

Evaluating:   4%|▍         | 191/4496 [00:00<00:07, 584.94pair/s]

Context: فاهتموا بتتبع حياة الرسول محمد) ( وسيرته واحاديثه وبحروب التحرير التي خاضها العرب المسلمون ضد الفرس والروم ،, Predicted Gloss: gloss.242, Correct Gloss: gloss.242, Ranking Score: 0.5601
Context: اما الحل من خلال الطريقة المختصرة وبافتراض وسط فرضي لقيم المتغير X (المبيعات ) مساوي الى 15 فان الحل الحسابي يوصف بالاتي, Predicted Gloss: gloss.3608, Correct Gloss: gloss.3608, Ranking Score: 0.2662
Context: - عبارة عن دائرة مقسمة الى مقاطع والتي تصف مكونات نسب المجتمع او العينة التي تعود الى اصناف مختلفة ،, Predicted Gloss: gloss.345, Correct Gloss: gloss.345, Ranking Score: 0.9225
Context: إلــى أن توقـف النـاس برهـة وراحـوا ينظـرون إليـه وقـد فوجئـوا باندفاعـه وبالطاقـة التـي تصـدر عنـه., Predicted Gloss: gloss.7464, Correct Gloss: gloss.7464, Ranking Score: 0.7349
Context: وفي الختام هذه رسالة أبلغوني إياها فأنا حرمت من إقامة المعارض بالخيام وأمامي اليوم خيمة, Predicted Gloss: gloss.14867, Correct Gloss: gloss.14867, Ranking Score: 0.8411
Context: ويُعد اغتصاب فلسطين عام 8491 م وق

Evaluating:   7%|▋         | 335/4496 [00:00<00:06, 665.02pair/s]

...

Context: تابع التلوين خارج المثلث بحيث تلون رأس أي مثلث - يشترك مع المثلث الأساسي بضلع مشترك - بلون مختلف عن لوني الرأسين المشتركين ., Predicted Gloss: gloss.6018, Correct Gloss: gloss.6008, Ranking Score: 0.4416
Context: ‮ ‬والمكوّنات الأساسيّة لهوّيّتها،, Predicted Gloss: gloss.10702, Correct Gloss: gloss.10703, Ranking Score: 0.2385
Context: - وزارة الشؤون الدينية والأوقاف وزارة مفصلية لأنها تتعلق بهموم المجتمع وأهمها الأمور الدينية والحياتية ومجالاتها الاجتماعية والثقافية والصحية وغيرها من المجالات وكثير من الناس يجهلون اهمية الوزارة وهي بمثابة سلاح مشاة وموجودين على الأرض في كافة الاتجاهات كالمساجد والمدارس وغيرها من مجالات الحياة ولكي تأخذ الوزارة مركزها الصحيح والفعلي لدينا المبررات لتكون سيادية., Predicted Gloss: gloss.1575, Correct Gloss: gloss.1575, Ranking Score: 0.5494
Context: فيظن أنه إنما أريد بها في تلك الصناعة ما قد تعود أن يفهمه عنها قبل شروعه في الصناعة., Predicted Gloss: gloss.5932, Correct Gloss: gloss.5932, Ranking Score: 0.8764
Context: كما أتقدم بخالص شكري وأمتن

Evaluating:  97%|█████████▋| 4383/4496 [00:07<00:00, 603.36pair/s]


Context: حيث أحب الرسم منذ الصغر وكانت هوايته التي نمت مع الوقت لتزهر أعمالا فنية رائعة, Predicted Gloss: gloss.7506, Correct Gloss: gloss.7506, Ranking Score: 0.7735
Context: مواد وطرائق البحث	25, Predicted Gloss: gloss.3916, Correct Gloss: gloss.3918, Ranking Score: 0.8509
Context: و أشارت المتحدثة في إفتتاح التظاهرة أن المجلس الأعلى للغة العربية من خلال اختيار إشكالية الملتقى يهدف إلى "الاستئناس بما كتب حول الدراسات البينية مستهديا بموضوعات هذا الحقل المعرفي الذي يسعى إلى جمع ما تشظى من العلوم والمعارف في محاولة منه لإعادة ربط جسور المعرفة فيما بينها لتكون اللغة نافذة العقل والروح"., Predicted Gloss: gloss.2399, Correct Gloss: gloss.2396, Ranking Score: 0.8017
Context: استعمال الراديو تشغيل الراديو., Predicted Gloss: gloss.6725, Correct Gloss: gloss.6725, Ranking Score: 0.8339
Context: ٤٦٠ نظام الدفع بأربع عجلات., Predicted Gloss: gloss.3100, Correct Gloss: gloss.3100, Ranking Score: 0.5365
Context: احياء الثقافة العربية الاسلامية والمحافظة على تراث الامة وحضارتها من خلال انشاء الم

Evaluating: 100%|██████████| 4496/4496 [00:07<00:00, 622.71pair/s]


Context: وإن كوننا كائنات بشرية أخلاقية يمنحنا حقوقًا خاصة بنا كحق الحرية والملكية،, Predicted Gloss: gloss.13519, Correct Gloss: gloss.13519, Ranking Score: 0.5850
Context: عداد القيم (عمود اليسار) إذا كانت قيمةِ الوسيطِ للعيّنةِ مُتضمّنةُ بالتّسلسل ،, Predicted Gloss: gloss.2928, Correct Gloss: gloss.2928, Ranking Score: 0.7527
Context: قامت اليوم شركة مايكروسوفت بإزاحة الستار عن تطبيق جديد لمستخدمي نظام التشغيل ويندوز 10 يحمل اسم Your Phone ،, Predicted Gloss: gloss.2428, Correct Gloss: gloss.2428, Ranking Score: 0.7258
Context: هـل التهريـب قريـبٌ منـا ؟, Predicted Gloss: gloss.13122, Correct Gloss: gloss.13122, Ranking Score: 0.3206
Context: هـل كنـت قبـل عـام أفكـر أن ابنتـي سـتعيش فـي النمسـا،, Predicted Gloss: gloss.8495, Correct Gloss: gloss.8495, Ranking Score: 0.6793
Context: ومن ثمّ رفع القطع من الزيت ونقلها على ورق المطبخ وتركها إلى أن تصبح باردة., Predicted Gloss: gloss.8439, Correct Gloss: gloss.8440, Ranking Score: 0.9469
Context: تطلـع حواليـه،, Predicted Gloss: gloss

**In the code bellow the predict_gloss function takes two parameters:**


**word_context:** This represents the context in which a word appears. In the evaluate_on_test_with_progress_and_debug function, the word_context is obtained from each entry in the test_data. Specifically, it is retrieved using word_context = entry['context'].

This suggests that the test_data is a list or iterable of dictionaries, where each dictionary represents an entry or instance in the test set. The 'context' key in each dictionary holds the actual context or surrounding text in which the word appears.

**glosses:** This is a list of possible meanings or definitions (glosses) for the word being disambiguated.

In the evaluate_on_test_with_progress_and_debug function, the possible_glosses are obtained from the wsd_dict dictionary using the lemma_id of each entry.

The line **possible_glosses = wsd_dict.get(lemma_id, [])** retrieves the list of glosses associated with the lemma_id from the wsd_dict dictionary.

If no glosses are found for a particular lemma_id, an **empty list []** is returned as a default value.


The **wsd_dict **dictionary is likely loaded or constructed from a separate file or data source that contains the mapping between lemma IDs and their corresponding glosses. The structure of wsd_dict is expected to be as follows:

In [ ]:
# Predict gloss for each word context and add debug prints
def predict_gloss(word_context, glosses):
    context_emb = extract_embeddings_with_cache(word_context)
    scores = []
    for gloss in glosses:
        gloss_emb = extract_embeddings_with_cache(gloss['gloss'])
        concatenated = np.concatenate((context_emb, gloss_emb))
        score = clf.predict_proba([concatenated])[0][1]
        scores.append((gloss['gloss_id'], score))
    predicted_gloss_id, _ = max(scores, key=lambda x: x[1])
    return predicted_gloss_id

# Function to evaluate the model on the test data with debug prints
def evaluate_on_test_with_progress_and_debug(test_data, wsd_dict):
    correct = 0
    total = len(test_data)
    start_time = time.time()
    last_update_time = start_time

    for i, entry in enumerate(tqdm(test_data, desc="Evaluating", unit="pair")):
        word_context = entry['context']
        lemma_id = entry['lemma_id']
        possible_glosses = wsd_dict.get(lemma_id, [])
        if not possible_glosses:
            continue
        predicted_gloss_id = predict_gloss(word_context, possible_glosses)
        correct_gloss_id = entry.get('gloss_id')
        if predicted_gloss_id == correct_gloss_id:
            correct += 1
        else:
            print(f"Context: {word_context}, Predicted Gloss: {predicted_gloss_id}, Correct Gloss: {correct_gloss_id}")

        current_time = time.time()
        if current_time - last_update_time >= 60:
            elapsed = current_time - start_time
            progress = (i + 1) / total
            remaining = elapsed / progress - elapsed
            tqdm.write(
                f"Progress: {progress * 100:.2f}% - Elapsed: {elapsed:.2f}s - Remaining: {remaining:.2f}s"
            )
            last_update_time = current_time

    accuracy = correct / total if total else 0
    elapsed_time = time.time() - start_time
    print(f"Evaluation Completed - Elapsed Time: {elapsed_time:.2f}s")
    return accuracy

# Evaluate on the test data with debug prints
test_accuracy = evaluate_on_test_with_progress_and_debug(test_data, wsd_dict)
print(f"Test Accuracy: {test_accuracy:.4f}")

Evaluating:   2%|▏         | 84/4496 [00:00<00:05, 836.22pair/s]

Context: حالة موقع تحريج كفردبيل- ريف جبلة ", Predicted Gloss: gloss.15993, Correct Gloss: gloss.15992
Context: وليس للقانون أثر رجعي., Predicted Gloss: gloss.14893, Correct Gloss: gloss.14888
Context: بحسب رئيس جمعية مزارعي فزان " عثمان الطاهر" حيث وصل إنتاج ليبيا من القمح والشعير خلال العام الماضي إلى (100 ) ألف طن،, Predicted Gloss: gloss.4777, Correct Gloss: gloss.4775
Context: - الدرجة مكونة من مرتبتين (احاد وعشرات) مرتبة العشرات تاخذ القيم (6,7,8,9) فمرتبة (6) ظهرت مرة واحدة لان كل درجات مرتبة عشرات (6) اكبر من (65) ،, Predicted Gloss: gloss.5874, Correct Gloss: gloss.5873
Context: حيث يذكر في الباب الأول (حقوق الشعب) نصوصاً قانونية تتعلق بالجنسية العراقية،, Predicted Gloss: gloss.1599, Correct Gloss: gloss.1590
Context: وأضاف ان الشركة عندما بدأت أعمالها كان اهتمامها يقتصر على الأدوات المنزلية الكهربائية من برادات وغسالات وأفران ومكانس كهربائية., Predicted Gloss: gloss.2782, Correct Gloss: gloss.2784
Context: والتي هي من أهم مبادئ حقوق الانسان في وقتنا الحاضر واعادة تعمير ماخربه

Evaluating:   6%|▌         | 264/4496 [00:00<00:05, 829.19pair/s]


Context: مما يُحدث التهابًا بالكبد وضعف قدرته على القيام بوظائفه., Predicted Gloss: gloss.1237, Correct Gloss: gloss.1235
Context: مقارنةً بالخلايا الكيراتينية التي تُشكل جُريبات الشعر،, Predicted Gloss: gloss.3483, Correct Gloss: gloss.3481
Context: بالنسبة لانتشار النوع الفرعي الرابع الوردية العينية ( (OR فهو غير معروف بسبب عدم خصوصية التظاهرات السريرية, وقد ارتبطت التظاهرات السريرية العينية مع التظاهرات الجلدية في الوردية في 33 ٪ من 100 مريض وردية جلدية في اليونان, وسجل حدوث الوردية العينية في المملكة المتحدة إلى 1., Predicted Gloss: gloss.2639, Correct Gloss: gloss.2636
Context: وعرفه (جوفمان) على أنه "كل نشاط الفرد الذي يحدث خلال فترة ما،, Predicted Gloss: gloss.812, Correct Gloss: gloss.814
Context: 2-	التاريخ يقوي اعتزاز المواطن بأمته ووطنه وبشهداء هذه الأمة والابطال المدافعين عنها., Predicted Gloss: gloss.7102, Correct Gloss: gloss.7100
Context: شبكة توليدية وشبكة تمييزية،, Predicted Gloss: gloss.3278, Correct Gloss: gloss.3271
Context: وعـدت إلـى الأهـل أخيـرًاً ” قـال نيكـول

Evaluating:  10%|█         | 459/4496 [00:00<00:04, 914.24pair/s]

...

Context: لكن ومن خلال استخدام البيانات المصطنعة المدربة على مجموعات بيانات أكبر بكثير،, Predicted Gloss: gloss.3013, Correct Gloss: gloss.3012
Context: ولذلك فقد اتفق على أن تستخدم في حسابها وحدة خاصة هي السنة الضوئية Light Year وهي المسافة التي يقطعها الضوء "وسرعته 300 ألف كيلو متر في الثانية" في سنة كاملة،, Predicted Gloss: gloss.11242, Correct Gloss: gloss.11246
Context: ‮ ‬حتّى الحروب التّحريريّة من الاستعمار،, Predicted Gloss: gloss.8471, Correct Gloss: gloss.8472
Context: هـو “شـعلة الحطـب” برنـار،, Predicted Gloss: gloss.10345, Correct Gloss: gloss.10344
Context: الدستور حسبما ورد في تقرير اللجنة الوطنية للإحصاء., Predicted Gloss: gloss.8858, Correct Gloss: gloss.8857
Context: التقويم يعني بيان السنة والشهر واليوم وضبطها ., Predicted Gloss: gloss.8170, Correct Gloss: gloss.8169
Context: ومـن يريـد أن يطالـع مـا يكتـب عـن مشـاهد القتـال المتواصـل., Predicted Gloss: gloss.1864, Correct Gloss: gloss.1861
Context: لا يمكن منح الكائنات الحية حريات طبيعية مطلقة دون ضوابط, Predicted Gl

Evaluating:  99%|█████████▉| 4442/4496 [00:10<00:00, 279.47pair/s]


Context: قـوات بقيـادة قائـد جيـش منطقـة قـازان سـتولنيك إيفـان موراشـكين الـذي أعـدم مـن قبـض عليهـم،, Predicted Gloss: gloss.9010, Correct Gloss: gloss.9009
Context: كان يتكلم بالرأى فيخطئ ويصيب ., Predicted Gloss: gloss.432, Correct Gloss: gloss.431
Context: طريـق الحريـر ., Predicted Gloss: gloss.14707, Correct Gloss: gloss.14701
Context: كبلاد الهند التي تضمّ مجموعةً من العادات الغريبة التي تسجّلها أعين الكاميرات،, Predicted Gloss: gloss.2774, Correct Gloss: gloss.2772
Context: الهدف هو إنشاء منتجات وعمليات وسياسات – طرق جديدة للحياة – تتكيف بشكل جيد مع الحياة على الأرض على المدى الطويل., Predicted Gloss: gloss.5449, Correct Gloss: gloss.5448
Context: بنغازي 27 فبراير 2022 م ( وال ) - تتجه أنظار العالم هذه الأيام نحو الأزمة التي اندلعت بين روسيا وأوكرانيا والتوترات الحاصلة في المنطقة إثر الاجتياح الروسي للأراضي الأوكرانية والتي تنذر بوقوع أزمات اقتصادية في العالم أجمع ،, Predicted Gloss: gloss.2983, Correct Gloss: gloss.2982
Context: 470), وهذا الفهم في كثير من الأحيان لا يشمل ال

Evaluating: 100%|██████████| 4496/4496 [00:10<00:00, 439.19pair/s]

Context: وتطال أعناقهم إلى تواريخ الأمم الخالية،, Predicted Gloss: gloss.10992, Correct Gloss: gloss.10988
Context: وفــي مطلــع الثلاثينــات نشــرت أول روايــة شيشــانية بقلــم ســعيد بــادروف., Predicted Gloss: gloss.1599, Correct Gloss: gloss.1590
Context: ذلك إلى أن يبلغ الملك سن الرشد،, Predicted Gloss: gloss.9745, Correct Gloss: gloss.9743
Context: وباعتباره وسيلة الأنظمة التعليمية لتشكيل الخبرات التربوية؛, Predicted Gloss: gloss.10674, Correct Gloss: gloss.10664
Context: وعرفه (أبراهامز) عندما قدم النظرية البلاغية للفولكلور:, Predicted Gloss: gloss.5543, Correct Gloss: gloss.5538
Context: وهي صفات للاستجابة الثنائية والتي توضح بصورة ادق بالمثال التالي ., Predicted Gloss: gloss.5332, Correct Gloss: gloss.5328
Context: ان متغير بيانات الاستجابه الثنائية (binary response) يمتلك حالتين فقط مثل وجود او عدم الصفة ،, Predicted Gloss: gloss.5332, Correct Gloss: gloss.5328
Evaluation Completed - Elapsed Time: 10.25s
Test Accuracy: 0.6815


In [ ]:
import json

def predict_all_glosses(word_context, glosses):
    context_emb = extract_embeddings_with_cache(word_context)
    results = []
    for gloss in glosses:
        gloss_emb = extract_embeddings_with_cache(gloss['gloss'])
        concatenated = np.concatenate((context_emb, gloss_emb))
        score = clf.predict_proba([concatenated])[0][1]
        results.append({
            "conetext_id": word_context,  # This should be changed to correct key if context ID is different
            "gloss_id": gloss['gloss_id'],
            "ranking_score": score
        })
    return results

def generate_results_json(test_data, wsd_dict):
    all_results = []
    for entry in test_data:
        word_context = entry['context_id']
        lemma_id = entry['lemma_id']
        possible_glosses = wsd_dict.get(lemma_id, [])
        if not possible_glosses:
            continue
        context_results = predict_all_glosses(word_context, possible_glosses)
        all_results.extend(context_results)

    with open('resultWSD2.json', 'w', encoding='utf-8') as f:
        json.dump(all_results, f, ensure_ascii=False, indent=4)

# Assuming test_data and wsd_dict are already loaded
generate_results_json(test_data, wsd_dict)